# Content Version Tracking System - Implementation Guide

This notebook implements the version tracking system described in `Docs/README_VERSION_TRACKING.md`.

## Overview

This system:
- Tracks content evolution across versions
- Links FAQs to specific content versions
- Auto-deprecates FAQs when content changes
- Provides full audit trail

**Key Concept:** The `content_version_history` table stores **metadata only** (checksums, version numbers, change summaries). Actual content lives in markdown files on disk.

## Prerequisites

⚠️ **IMPORTANT**: You must run the following notebooks in order:
1. `setup_databases.ipynb` - Creates all databases
2. `daily_version_tracking_pipeline.ipynb` - Populates version history data
3. This notebook - Queries and analyzes version data

## Step 1: Setup and Imports

In [6]:
import pandas as pd
import sqlite3
import json
from pathlib import Path
from datetime import datetime
import hashlib

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

print("✅ Imports successful")

✅ Imports successful


## Step 2: Connect to Databases

Load data from the 4 databases created by `setup_databases.ipynb`.

In [ ]:
# Define paths
DOCS_PATH = Path('../Docs')
DATABASES_DIR = Path('../databases')

# Database paths
CONTENT_REPO_DB = DATABASES_DIR / 'content_repo.db'
FAQ_QUESTION_DB = DATABASES_DIR / 'faq_question.db'
FAQ_ANSWER_DB = DATABASES_DIR / 'faq_answer.db'
VERSION_TRACKING_DB = DATABASES_DIR / 'version_tracking.db'

# Load content_repo from database
print(f"📊 Loading data from databases...\n")

with sqlite3.connect(CONTENT_REPO_DB) as conn_content:
    content_repo = pd.read_sql("SELECT * FROM content_repo", conn_content)
print(f"  - content_repo: {len(content_repo)} records")

# Load FAQ questions from database
with sqlite3.connect(FAQ_QUESTION_DB) as conn_question:
    faq_questions = pd.read_sql("SELECT * FROM faq_questions", conn_question)
print(f"  - faq_questions: {len(faq_questions)} records")

# Load FAQ answers from database
with sqlite3.connect(FAQ_ANSWER_DB) as conn_answer:
    faq_answers = pd.read_sql("SELECT * FROM faq_answers", conn_answer)
print(f"  - faq_answers: {len(faq_answers)} records")

print(f"\n✅ All data loaded from databases")

# Preview content_repo
print("\n📄 Sample content_repo records:")
content_repo[['ud_source_file_id', 'raw_file_nme', 'raw_file_page_nbr', 'version_nbr', 'file_status']].head(6)

## Step 5: Load Version Tracking Data

**IMPORTANT**: This notebook is READ-ONLY. It does not create or modify database data. All version tracking data must be populated by running `daily_version_tracking_pipeline.ipynb` first.

If the database is empty, the notebook will show examples with empty DataFrames and guide you to run the daily pipeline.

In [ ]:
# Verify databases exist
print("🔍 Verifying databases...\n")
missing_dbs = []
for db_path in [CONTENT_REPO_DB, FAQ_QUESTION_DB, FAQ_ANSWER_DB, VERSION_TRACKING_DB]:
    if not db_path.exists():
        missing_dbs.append(db_path.name)
        print(f"  ❌ {db_path.name} - NOT FOUND")
    else:
        print(f"  ✅ {db_path.name} - Found")

if missing_dbs:
    raise FileNotFoundError(
        f"Missing databases: {', '.join(missing_dbs)}\n"
        f"Please run setup_databases.ipynb first to create all databases."
    )

# Verify schema exists using temporary connection
with sqlite3.connect(VERSION_TRACKING_DB) as conn_verify:
    cursor = conn_verify.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='content_version_history'")
    
    if cursor.fetchone() is None:
        raise RuntimeError(
            f"Schema not found in {VERSION_TRACKING_DB}\n"
            f"Please run setup_databases.ipynb to apply the schema."
        )
    
    # Verify all tables
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn_verify)
    print(f"\n✅ Connected to version_tracking.db")
    print(f"📋 Available tables: {', '.join(tables['name'].tolist())}")

## Step 4: Load Version Tracking Data

Load version tracking data from database. If empty, load sample data for demonstration.

In [ ]:
# Load version tracking data from database (populated by daily pipeline)
print("📊 Loading version tracking data from database...\n")

try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_version:
        content_version_history = pd.read_sql("SELECT * FROM content_version_history", conn_version)
    print(f"  - content_version_history: {len(content_version_history)} records")
except Exception as e:
    content_version_history = pd.DataFrame()
    print(f"  - content_version_history: Error loading ({e})")

try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_link:
        faq_content_link = pd.read_sql("SELECT * FROM faq_content_link", conn_link)
    print(f"  - faq_content_link: {len(faq_content_link)} records")
except Exception as e:
    faq_content_link = pd.DataFrame()
    print(f"  - faq_content_link: Error loading ({e})")

try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_validation:
        faq_validation_history = pd.read_sql("SELECT * FROM faq_validation_history", conn_validation)
    print(f"  - faq_validation_history: {len(faq_validation_history)} records")
except Exception as e:
    faq_validation_history = pd.DataFrame()
    print(f"  - faq_validation_history: Error loading ({e})")

try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_impact:
        content_change_impact = pd.read_sql("SELECT * FROM content_change_impact", conn_impact)
    print(f"  - content_change_impact: {len(content_change_impact)} records")
except Exception as e:
    content_change_impact = pd.DataFrame()
    print(f"  - content_change_impact: Error loading ({e})")

print(f"\n✅ Version tracking data loaded")

# Check if data exists
if len(content_version_history) > 0:
    # Preview version history
    print("\n🔍 Sample version history (showing version chain):")
    display(content_version_history[[
        'version_history_id', 'content_id', 'previous_version_id',
        'version_sequence', 'file_name', 'page_number', 'change_type'
    ]].head(10))
else:
    print("\n⚠️ No version history data available.")
    print("📌 To populate data, run: daily_version_tracking_pipeline.ipynb")
    print("   This will detect content changes and create version records.")

## Step 6: Data Availability Check

Before proceeding with examples, verify that version tracking data exists in the database.

In [ ]:
# Check if version tracking data exists
with sqlite3.connect(VERSION_TRACKING_DB) as conn_check:
    cursor = conn_check.cursor()
    cursor.execute("SELECT COUNT(*) as count FROM content_version_history")
    count = cursor.fetchone()[0]

if count == 0:
    print("⚠️ WARNING: No version history data found in database\n")
    print("This notebook demonstrates version tracking concepts, but requires data to be populated first.\n")
    print("📌 Next Steps:")
    print("  1. Run daily_version_tracking_pipeline.ipynb to detect content changes")
    print("  2. This will create initial version records (version_sequence=1) for all content")
    print("  3. Re-run this notebook to explore the version chains\n")
    print("⏭️ You can continue through this notebook to see the structure and queries,")
    print("   but most examples will show empty results until data is populated.\n")
else:
    print(f"✅ Database contains {count} version history records")
    print("   Ready to explore version tracking data!\n")
    
    # Show summary
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_summary:
        latest_count = pd.read_sql("SELECT COUNT(*) as count FROM content_version_history WHERE is_latest_version = 1", conn_summary).iloc[0]['count']
        faq_link_count = pd.read_sql("SELECT COUNT(*) as count FROM faq_content_link", conn_summary).iloc[0]['count']
    
    print(f"📊 Data Summary:")
    print(f"  - Total versions: {count}")
    print(f"  - Latest versions: {latest_count}")
    print(f"  - FAQ links: {faq_link_count}")

## Step 7: Understanding the Data Model

### Key Insight: Self-Referential FK Design

The `previous_version_id` creates a linked list **within** the `content_version_history` table.

**NOTE**: The following examples will only show results if version history data exists. Run the daily pipeline first to populate data.

In [ ]:
# Example: Trace version chain for a specific file and page
# This example shows the structure - adapt to your actual data

if len(content_version_history) > 0:
    # Get a sample file and page to demonstrate
    sample_file = content_version_history.iloc[0]['file_name']
    sample_page = content_version_history.iloc[0]['page_number']
    
    print(f"📜 Version Chain for '{sample_file}' Page {sample_page}:\n")
    
    file_versions = content_version_history[
        (content_version_history['file_name'] == sample_file) &
        (content_version_history['page_number'] == sample_page)
    ].sort_values('version_sequence')
    
    for _, v in file_versions.iterrows():
        print(f"Version {v['version_sequence']}:")
        print(f"  - version_history_id: {v['version_history_id']}")
        print(f"  - content_id: {v['content_id']}")
        print(f"  - previous_version_id: {v['previous_version_id']}")
        print(f"  - change_type: {v['change_type']}")
        print(f"  - change_summary: {v['change_summary']}")
        print(f"  - is_latest: {v['is_latest_version']}")
        print()
    
    print("\n💡 Notice:")
    print("  - Version 1: previous_version_id = NaN or NULL (no previous version)")
    if len(file_versions) > 1:
        print(f"  - Version 2+: previous_version_id points to previous version_history_id")
        print("  - This creates a linked list: v1 ← v2 ← v3 ...")
else:
    print("⚠️ No version history data available to demonstrate.")
    print("   Run daily_version_tracking_pipeline.ipynb to populate data.")

## Step 8: Helper Functions

Implement the helper functions from README.

In [ ]:
def get_full_version_chain(version_history_id: int, df: pd.DataFrame):
    """
    Get the complete version chain for a given version.
    
    Args:
        version_history_id: The version_history_id to start from
        df: The content_version_history DataFrame
    
    Returns:
        List of version records in chronological order (v1, v2, v3, ...)
    """
    versions = []
    
    # Start with the given version
    current_version = df[df['version_history_id'] == version_history_id].iloc[0]
    versions.append(current_version)
    
    # Walk backwards through the chain
    while pd.notna(current_version['previous_version_id']):
        prev_id = int(current_version['previous_version_id'])
        current_version = df[df['version_history_id'] == prev_id].iloc[0]
        versions.append(current_version)
    
    # Reverse to get chronological order
    return versions[::-1]


def build_faq_details(faq_links, faq_questions_df, faq_answers_df):
    """
    Build Q&A pairs from faq_content_link records.
    """
    qa_pairs = []
    
    for _, link in faq_links.iterrows():
        question_match = faq_questions_df[faq_questions_df['question_id'] == link['question_id']]
        if len(question_match) == 0:
            continue
        question = question_match.iloc[0]
        
        answer_txt = None
        if pd.notna(link['answer_id']):
            answer_match = faq_answers_df[faq_answers_df['answer_id'] == link['answer_id']]
            if len(answer_match) > 0:
                answer = answer_match.iloc[0]
                answer_txt = answer['faq_answer_txt']
        
        qa_pairs.append({
            'question_id': link['question_id'],
            'question': question['question_txt'],
            'answer_id': link['answer_id'],
            'answer': answer_txt,
            'is_valid': link['is_currently_valid'],
            'deprecated_reason': link.get('deprecated_reason'),
            'confidence_score': link.get('confidence_score')
        })
    
    return qa_pairs


print("✅ Helper functions defined")

## Step 9: Test Full Version Chain Traversal

**NOTE**: This requires version history data to exist.

In [ ]:
# Get full version chain (requires data to exist)
if len(content_version_history) > 0:
    # Find a version to trace (use any version with version_sequence > 1 if available)
    multi_version = content_version_history[content_version_history['version_sequence'] > 1]
    
    if len(multi_version) > 0:
        sample_version_id = multi_version.iloc[0]['version_history_id']
        print(f"🔗 Full Version Chain for version_history_id={sample_version_id}:\n")
        
        versions = get_full_version_chain(sample_version_id, content_version_history)
        
        for i, v in enumerate(versions, 1):
            print(f"Version {i}:")
            print(f"  - version_history_id: {v['version_history_id']}")
            print(f"  - content_id: {v['content_id']}")
            print(f"  - version_sequence: {v['version_sequence']}")
            print(f"  - change_type: {v['change_type']}")
            print(f"  - change_summary: {v['change_summary']}")
            print(f"  - checksum: {v['content_checksum'][:16]}...")
            print()
        
        print(f"✅ Successfully traversed {len(versions)} versions")
    else:
        # Only v1 exists, use that
        sample_version_id = content_version_history.iloc[0]['version_history_id']
        print(f"🔗 Full Version Chain for version_history_id={sample_version_id}:\n")
        
        versions = get_full_version_chain(sample_version_id, content_version_history)
        
        for i, v in enumerate(versions, 1):
            print(f"Version {i}:")
            print(f"  - version_history_id: {v['version_history_id']}")
            print(f"  - content_id: {v['content_id']}")
            print(f"  - version_sequence: {v['version_sequence']}")
            print(f"  - change_type: {v['change_type']}")
            print(f"  - change_summary: {v['change_summary']}")
            print()
        
        print("📌 Note: Only version 1 exists. Run pipeline multiple times to see version chains.")
else:
    print("⚠️ No version history data available.")
    print("   Run daily_version_tracking_pipeline.ipynb to populate data.")

## Step 10: Query Examples

### 10.1 Get All Current Valid FAQs for a File+Page

**NOTE**: Results depend on data existing in the database.

In [ ]:
# Query current valid FAQs
if len(content_version_history) > 0:
    # Use actual file name from data
    sample_file = content_version_history.iloc[0]['file_name']
    sample_page = content_version_history.iloc[0]['page_number']
    
    query = f"""
    SELECT
        fcl.question_id,
        fcl.answer_id,
        cvh.version_sequence,
        cvh.file_name,
        cvh.page_number,
        cvh.change_type,
        fcl.is_currently_valid,
        fcl.confidence_score
    FROM faq_content_link fcl
    JOIN content_version_history cvh ON fcl.version_history_id = cvh.version_history_id
    WHERE cvh.file_name = '{sample_file}'
      AND cvh.page_number = {sample_page}
      AND cvh.is_latest_version = 1
      AND fcl.is_currently_valid = 1
    """
    
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_query:
        current_faqs = pd.read_sql(query, conn_query)
    
    print(f"📌 Current Valid FAQs for '{sample_file}' Page {sample_page}:\n")
    
    if len(current_faqs) > 0:
        print(current_faqs)
        
        # Get full Q&A text
        print(f"\n📝 Full Q&A Details:\n")
        for _, row in current_faqs.iterrows():
            q = faq_questions[faq_questions['question_id'] == row['question_id']]
            if len(q) > 0:
                print(f"Q: {q.iloc[0]['question_txt']}")
                
                if pd.notna(row['answer_id']):
                    a = faq_answers[faq_answers['answer_id'] == row['answer_id']]
                    if len(a) > 0:
                        print(f"A: {a.iloc[0]['faq_answer_txt'][:200]}...")
                print(f"Version: {row['version_sequence']}, Confidence: {row['confidence_score']}")
                print()
    else:
        print("No FAQs found for this file+page.")
        print("FAQs need to be linked to content versions separately.")
else:
    print("⚠️ No version history data available.")
    print("   Run daily_version_tracking_pipeline.ipynb to populate data.")

### 10.2 Get Version History for a Specific Page

In [ ]:
# Query version history for a specific page
if len(content_version_history) > 0:
    # Use actual file name from data
    sample_file = content_version_history.iloc[0]['file_name']
    sample_page = content_version_history.iloc[0]['page_number']
    
    query = f"""
    SELECT
        cvh.version_sequence,
        cvh.content_id,
        cvh.content_checksum,
        cvh.change_type,
        cvh.change_summary,
        cvh.version_created_at,
        COUNT(fcl.link_id) as faq_count
    FROM content_version_history cvh
    LEFT JOIN faq_content_link fcl ON cvh.version_history_id = fcl.version_history_id
    WHERE cvh.file_name = '{sample_file}'
      AND cvh.page_number = {sample_page}
    GROUP BY cvh.version_history_id
    ORDER BY cvh.version_sequence
    """
    
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_query:
        version_history = pd.read_sql(query, conn_query)
    
    print(f"📜 Version History for '{sample_file}' Page {sample_page}:\n")
    print(version_history)
else:
    print("⚠️ No version history data available.")
    print("   Run daily_version_tracking_pipeline.ipynb to populate data.")

### 9.3 Find FAQs That Need Regeneration

In [ ]:
query = """
SELECT
    fcl.link_id,
    fcl.question_id,
    cvh.file_name,
    cvh.page_number,
    cvh.version_sequence,
    cvh.change_type,
    cvh.change_summary,
    fcl.deprecated_reason,
    fcl.deprecated_at
FROM faq_content_link fcl
JOIN content_version_history cvh ON fcl.version_history_id = cvh.version_history_id
WHERE fcl.is_currently_valid = 0
  AND fcl.superseded_by_link_id IS NULL
  AND fcl.deprecated_reason = 'content_updated'
"""

with sqlite3.connect(VERSION_TRACKING_DB) as conn_query:
    needs_regen = pd.read_sql(query, conn_query)

print("⚠️ FAQs That Need Regeneration:\n")
print(needs_regen)

if len(needs_regen) > 0:
    print("\n💡 These FAQs were deprecated due to content updates and need to be regenerated.")

### 9.4 Side-by-Side Version Comparison

In [ ]:
query = """
WITH current_version AS (
    SELECT cvh.*
    FROM content_version_history cvh
    WHERE cvh.file_name = 'TCI FEDERAL PTO POLICY.pdf'
      AND cvh.page_number = 1
      AND cvh.is_latest_version = 1
),
previous_version AS (
    SELECT prev_cvh.*
    FROM current_version cv
    JOIN content_version_history prev_cvh ON cv.previous_version_id = prev_cvh.version_history_id
)
SELECT
    cv.file_name,
    cv.page_number,
    
    pv.version_sequence as prev_version_number,
    pv.content_id as prev_content_id,
    pv.content_checksum as prev_checksum,
    pv.change_summary as prev_change_summary,
    
    cv.version_sequence as current_version_number,
    cv.content_id as current_content_id,
    cv.content_checksum as current_checksum,
    cv.change_summary as current_change_summary,
    
    cv.change_type
FROM current_version cv
LEFT JOIN previous_version pv ON 1=1
"""

with sqlite3.connect(VERSION_TRACKING_DB) as conn_query:
    comparison = pd.read_sql(query, conn_query)

print("🔄 Side-by-Side Version Comparison:\n")

if len(comparison) > 0:
    row = comparison.iloc[0]
    print(f"File: {row['file_name']}, Page: {row['page_number']}\n")
    
    print(f"Previous Version {row['prev_version_number']}:")
    print(f"  - Content ID: {row['prev_content_id']}")
    print(f"  - Checksum: {row['prev_checksum'][:16]}...")
    print(f"  - Summary: {row['prev_change_summary']}")
    print()
    
    print(f"Current Version {row['current_version_number']}:")
    print(f"  - Content ID: {row['current_content_id']}")
    print(f"  - Checksum: {row['current_checksum'][:16]}...")
    print(f"  - Summary: {row['current_change_summary']}")
    print()
    
    print(f"Change Type: {row['change_type'].upper()}")

## Step 10: Implement the Version Tracking Pipeline

This pipeline compares **adjacent versions** (v2 vs v1, v3 vs v2).

In [ ]:
def version_tracking_pipeline(last_modified_dt: str, 
                              content_repo_df, 
                              content_version_history_df,
                              faq_content_link_df,
                              faq_questions_df,
                              faq_answers_df):
    """
    Track content versions and FAQs for updated content.
    
    NOTE: This pipeline compares ADJACENT versions (v2 vs v1, v3 vs v2).
    To get full version history (v1 → v2 → v3), use get_full_version_chain().
    """
    
    # 1. Find updated content from content_repo
    updated_content = content_repo_df[
        content_repo_df['last_modified_dt'] > last_modified_dt
    ]
    
    print(f"🔍 Found {len(updated_content)} updated content records since {last_modified_dt}\n")
    
    # 2. For each updated content
    for _, content in updated_content.iterrows():
        
        # 3. Look up version history for this content
        current_version_match = content_version_history_df[
            content_version_history_df['content_id'] == content['ud_source_file_id']
        ]
        
        if len(current_version_match) == 0:
            continue
            
        current_version_record = current_version_match.iloc[0]
        
        # 4. Get previous version (if exists) by following previous_version_id
        previous_version_record = None
        previous_content = None
        
        if pd.notna(current_version_record['previous_version_id']):
            # Get the previous version history record
            prev_id = int(current_version_record['previous_version_id'])
            previous_version_record = content_version_history_df[
                content_version_history_df['version_history_id'] == prev_id
            ].iloc[0]
            
            # Get the actual content from content_repo
            previous_content = content_repo_df[
                content_repo_df['ud_source_file_id'] == previous_version_record['content_id']
            ].iloc[0]
            
            # Get previous version's FAQs
            previous_faqs = faq_content_link_df[
                (faq_content_link_df['version_history_id'] == previous_version_record['version_history_id']) &
                (faq_content_link_df['is_currently_valid'] == False)
            ]
        else:
            previous_faqs = pd.DataFrame()
        
        # 5. Get current version's FAQs
        current_faqs = faq_content_link_df[
            (faq_content_link_df['version_history_id'] == current_version_record['version_history_id']) &
            (faq_content_link_df['is_currently_valid'] == True)
        ]
        
        # 6. Build comparison view
        comparison = {
            'file_name': current_version_record['file_name'],
            'page_number': current_version_record['page_number'],
            
            'previous_version': {
                'version_history_id': previous_version_record['version_history_id'] if previous_version_record is not None else None,
                'content_id': previous_version_record['content_id'] if previous_version_record is not None else None,
                'version_number': previous_version_record['version_sequence'] if previous_version_record is not None else None,
                'markdown_file_path': previous_content['extracted_markdown_file_path'] if previous_content is not None else None,
                'checksum': previous_version_record['content_checksum'] if previous_version_record is not None else None,
                'faqs': build_faq_details(previous_faqs, faq_questions_df, faq_answers_df)
            },
            
            'current_version': {
                'version_history_id': current_version_record['version_history_id'],
                'content_id': content['ud_source_file_id'],
                'version_number': current_version_record['version_sequence'],
                'markdown_file_path': content['extracted_markdown_file_path'],
                'checksum': current_version_record['content_checksum'],
                'faqs': build_faq_details(current_faqs, faq_questions_df, faq_answers_df)
            },
            
            'changes': {
                'change_type': current_version_record['change_type'],
                'change_summary': current_version_record['change_summary'],
                'diff': json.loads(current_version_record['content_diff']) if pd.notna(current_version_record['content_diff']) and current_version_record['content_diff'] != '{}' else {}
            }
        }
        
        yield comparison

print("✅ Pipeline function defined")

## Step 11: Run the Pipeline

Execute the pipeline to find content updated after a specific date.

In [ ]:
# Run pipeline for content updated after September 1, 2025
results = list(version_tracking_pipeline(
    last_modified_dt="2025-09-01T00:00:00.000Z",
    content_repo_df=content_repo,
    content_version_history_df=content_version_history,
    faq_content_link_df=faq_content_link,
    faq_questions_df=faq_questions,
    faq_answers_df=faq_answers
))

print(f"\n📊 Pipeline Results: {len(results)} comparisons\n")
print("="*80)

for comparison in results:
    print(f"\n📄 File: {comparison['file_name']}, Page: {comparison['page_number']}")
    print("="*80)
    
    # Previous version
    if comparison['previous_version']['version_number'] is not None:
        print(f"\n📌 Previous Version {comparison['previous_version']['version_number']}:")
        print(f"  Content ID: {comparison['previous_version']['content_id']}")
        print(f"  Checksum: {comparison['previous_version']['checksum'][:16]}...")
        print(f"  FAQs: {len(comparison['previous_version']['faqs'])}")
        
        for faq in comparison['previous_version']['faqs']:
            print(f"    Q ({faq['question_id']}): {faq['question'][:80]}...")
            if faq['answer']:
                print(f"    A: {faq['answer'][:80]}...")
            print(f"    Valid: {faq['is_valid']}, Reason: {faq['deprecated_reason']}")
    else:
        print("\n📌 No previous version (this is version 1)")
    
    # Current version
    print(f"\n✅ Current Version {comparison['current_version']['version_number']}:")
    print(f"  Content ID: {comparison['current_version']['content_id']}")
    print(f"  Checksum: {comparison['current_version']['checksum'][:16]}...")
    print(f"  FAQs: {len(comparison['current_version']['faqs'])}")
    
    for faq in comparison['current_version']['faqs']:
        print(f"    Q ({faq['question_id']}): {faq['question'][:80]}...")
        if faq['answer']:
            print(f"    A: {faq['answer'][:80]}...")
        print(f"    Valid: {faq['is_valid']}, Confidence: {faq['confidence_score']}")
    
    # Changes
    print(f"\n🔄 Changes:")
    print(f"  Type: {comparison['changes']['change_type'].upper()}")
    print(f"  Summary: {comparison['changes']['change_summary']}")
    if comparison['changes']['diff']:
        print(f"  Diff: {json.dumps(comparison['changes']['diff'], indent=2)}")
    
    print("\n" + "="*80)

## Step 12: Advanced Usage - Full Version History

Get the complete version chain for a specific page.

In [ ]:
# Find latest version for PTO Policy page 1
latest = content_version_history[
    (content_version_history['file_name'] == 'TCI FEDERAL PTO POLICY.pdf') &
    (content_version_history['page_number'] == 1) &
    (content_version_history['is_latest_version'] == True)
].iloc[0]

print(f"🔗 Full Version History for '{latest['file_name']}' Page {latest['page_number']}")
print("="*80)

# Get full chain
all_versions = get_full_version_chain(latest['version_history_id'], content_version_history)

for i, v in enumerate(all_versions, 1):
    print(f"\nVersion {i} (Sequence: {v['version_sequence']}, ID: {v['version_history_id']})")
    print(f"  Content ID: {v['content_id']}")
    print(f"  Change Type: {v['change_type']}")
    print(f"  Summary: {v['change_summary']}")
    print(f"  Checksum: {v['content_checksum'][:16]}...")
    print(f"  Created: {v['version_created_at']}")
    print(f"  Is Latest: {v['is_latest_version']}")

print(f"\n✅ Total versions: {len(all_versions)}")

## Step 13: Impact Analysis

Analyze the impact of content changes on FAQs.

In [ ]:
query = """
SELECT
    cvh.file_name,
    cvh.page_number,
    cvh.version_sequence,
    cvh.change_type,
    cci.total_faqs_affected,
    cci.faqs_auto_deprecated,
    cci.faqs_flagged_for_review,
    cci.faqs_still_valid,
    cci.impact_severity,
    cci.manual_review_required
FROM content_change_impact cci
JOIN content_version_history cvh ON cci.version_history_id = cvh.version_history_id
ORDER BY cci.impact_severity DESC, cci.total_faqs_affected DESC
"""

with sqlite3.connect(VERSION_TRACKING_DB) as conn_query:
    impact = pd.read_sql(query, conn_query)

print("📊 Content Change Impact Analysis:\n")
print(impact)

if len(impact) > 0:
    print("\n💡 Summary:")
    print(f"  Total changes tracked: {len(impact)}")
    print(f"  High severity changes: {len(impact[impact['impact_severity'] == 'high'])}")
    print(f"  Changes requiring manual review: {len(impact[impact['manual_review_required'] == 1])}")
    print(f"  Total FAQs affected: {impact['total_faqs_affected'].sum()}")
    print(f"  FAQs auto-deprecated: {impact['faqs_auto_deprecated'].sum()}")

## Step 14: Create Simulation Data

Simulate creating a new version (Version 3) for demonstration.

In [ ]:
# This is a SIMULATION - not adding to actual database
print("🎭 Simulating Version 3 Creation for PTO Policy Page 1\n")

# Simulate new content in content_repo
new_content_id = 3010
new_checksum = hashlib.sha256("Updated PTO Policy - 20 hours/month".encode()).hexdigest()

# Simulate new version history record
new_version = {
    'version_history_id': 11,
    'content_id': new_content_id,
    'previous_version_id': 2,  # Points to version_history_id=2
    'version_sequence': 3,
    'is_latest_version': True,
    'file_name': 'TCI FEDERAL PTO POLICY.pdf',
    'page_number': 1,
    'content_checksum': new_checksum,
    'change_type': 'major',
    'change_summary': 'PTO accrual increased from 15 to 20 hours/month',
    'version_created_at': datetime.now().isoformat()
}

print("📝 New Version Record (Simulated):")
for key, value in new_version.items():
    print(f"  {key}: {value}")

print("\n🔗 Version Chain After v3:")
print("  v1 (id=1, content=100) ← v2 (id=2, content=3001) ← v3 (id=11, content=3010)")

print("\n💡 When v3 is created:")
print("  1. Set v2.is_latest_version = FALSE")
print("  2. Insert new version with previous_version_id = 2")
print("  3. Deprecate v2's FAQs (is_currently_valid = FALSE)")
print("  4. Generate new FAQs for v3")
print("  5. Create impact analysis record")

## Step 15: Validation and Data Quality Checks

In [ ]:
print("🔍 Data Quality Validation\n")
print("="*80)

# Check 1: Each file+page should have exactly one latest version
print("\n✓ Check 1: Latest Version Uniqueness")
latest_check = content_version_history[content_version_history['is_latest_version'] == True]
duplicates = latest_check.groupby(['file_name', 'page_number']).size()
duplicates = duplicates[duplicates > 1]

if len(duplicates) == 0:
    print("  ✅ PASS: Each file+page has exactly one latest version")
else:
    print(f"  ❌ FAIL: Found {len(duplicates)} file+page with multiple latest versions")
    print(duplicates)

# Check 2: Version sequences should be consecutive
print("\n✓ Check 2: Consecutive Version Sequences")
for (file_name, page_num), group in content_version_history.groupby(['file_name', 'page_number']):
    sequences = sorted(group['version_sequence'].tolist())
    expected = list(range(1, len(sequences) + 1))
    if sequences != expected:
        print(f"  ❌ FAIL: {file_name} page {page_num} has non-consecutive versions: {sequences}")
    else:
        print(f"  ✅ {file_name} page {page_num}: sequences OK {sequences}")

# Check 3: previous_version_id should point to valid version_history_id
print("\n✓ Check 3: Valid Foreign Keys")
invalid_fks = []
for _, row in content_version_history.iterrows():
    if pd.notna(row['previous_version_id']):
        prev_id = int(row['previous_version_id'])
        exists = content_version_history[content_version_history['version_history_id'] == prev_id]
        if len(exists) == 0:
            invalid_fks.append((row['version_history_id'], prev_id))

if len(invalid_fks) == 0:
    print("  ✅ PASS: All previous_version_id references are valid")
else:
    print(f"  ❌ FAIL: Found {len(invalid_fks)} invalid previous_version_id references")
    for vid, prev_id in invalid_fks:
        print(f"    version_history_id={vid} references non-existent previous_version_id={prev_id}")

# Check 4: No circular references
print("\n✓ Check 4: No Circular References")
circular = False
for _, row in content_version_history.iterrows():
    visited = set()
    current = row
    while pd.notna(current['previous_version_id']):
        vid = current['version_history_id']
        if vid in visited:
            print(f"  ❌ FAIL: Circular reference detected at version_history_id={vid}")
            circular = True
            break
        visited.add(vid)
        prev_id = int(current['previous_version_id'])
        current = content_version_history[content_version_history['version_history_id'] == prev_id].iloc[0]

if not circular:
    print("  ✅ PASS: No circular references detected")

print("\n" + "="*80)
print("✅ Validation Complete")

## Summary

This notebook demonstrated:

1. ✅ Loading and setting up the version tracking system
2. ✅ Understanding the self-referential FK design
3. ✅ Implementing helper functions for version traversal
4. ✅ Running SQL queries for common use cases
5. ✅ Executing the version tracking pipeline
6. ✅ Getting full version history
7. ✅ Analyzing change impact
8. ✅ Data quality validation

### Key Takeaways

- **Content is NOT in the database**: Version history stores metadata only. Actual content lives in markdown files.
- **Self-referential FK**: `previous_version_id` points to `version_history_id` (not `content_id`)
- **Adjacent comparison by default**: Pipeline compares v(n) vs v(n-1). Use `get_full_version_chain()` for complete history.
- **Unlimited versions**: The linked list design supports any number of versions (v1, v2, v3, ...).

### Next Steps

1. Integrate with content ingestion pipeline
2. Implement automated version detection on content updates
3. Build FAQ regeneration workflow for deprecated FAQs
4. Add visualization for version timelines
5. Implement content diff generation

In [ ]:
# All database connections are now properly closed using context managers
print("✅ All database connections have been properly managed with context managers")